In [1]:
import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [12]:
# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)
    
# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)
    
# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(
        target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    # Regression
    return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

In [2]:
path = "./data/"

In [4]:
filename_read = os.path.join(path,"Train_Solar.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

In [5]:
df.head(1)

,datetime,date,time,I,T,UV,WS,RH,P
0,1/1/2017 6:00,1/1/2017,6:00:00,0.0,24.6,0.0,1.2,59.8,0.0


In [7]:
# create feature vector
missing_median(df, 'I')
missing_median(df, 'T')
missing_median(df, 'UV')
missing_median(df, 'WS')
missing_median(df, 'P')
df.drop('datetime',1,inplace=True)
df.drop('date',1,inplace=True)
df.drop('time',1,inplace=True)

In [8]:
df.head(1)

,I,T,UV,WS,RH,P
0,0.0,24.6,0.0,1.2,59.8,0.0


In [11]:
# Drop outliers in horsepower
print("Length before P outliers dropped: {}".format(len(df)))
remove_outliers(df,'P',2)
print("Length after P outliers dropped: {}".format(len(df)))

Length before P outliers dropped: 131586
Length after P outliers dropped: 129577


In [14]:
x,y = to_xy(df,"P")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [15]:
print(x[0:5])

[[ 0.    24.6    0.     1.2   59.8  ]
 [ 0.    24.5    0.     1.4   60.   ]
 [ 0.    24.5    0.     1.9   60.2  ]
 [ 0.    24.5    0.     1.3   60.3  ]
 [ 0.    24.6    0.     1.4   60.125]]


In [16]:
print(y[0:5])

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]


In [17]:
# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(    
    x, y, test_size=0.25, random_state=42)

In [18]:
model = Sequential()
model.add(Dense(50, input_dim=x.shape[1], activation='relu'))
model.add(Dense(25,activation='relu'))
model.add(Dense(1)) # Output

In [19]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [20]:
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

In [22]:
checkpointer = ModelCheckpoint(filepath="R3_model.hdf5", verbose=0, save_best_only=True) # save best model

In [23]:
model.fit(x,y,validation_data=(x_test,y_test),callbacks=[monitor,checkpointer],verbose=2,epochs=1000)

Train on 129577 samples, validate on 32395 samples
Epoch 1/1000
 - 6s - loss: 1.5552 - val_loss: 0.2684
Epoch 2/1000
 - 5s - loss: 0.5589 - val_loss: 0.2605
Epoch 3/1000
 - 5s - loss: 0.4576 - val_loss: 0.2574
Epoch 4/1000
 - 4s - loss: 0.4391 - val_loss: 0.3998
Epoch 5/1000
 - 5s - loss: 0.3602 - val_loss: 0.2504
Epoch 6/1000
 - 5s - loss: 0.3573 - val_loss: 0.3400
Epoch 7/1000
 - 5s - loss: 0.3154 - val_loss: 0.2407
Epoch 8/1000
 - 4s - loss: 0.3028 - val_loss: 0.5103
Epoch 9/1000
 - 5s - loss: 0.2855 - val_loss: 0.2700
Epoch 10/1000
 - 5s - loss: 0.2746 - val_loss: 0.2409
Epoch 11/1000
 - 5s - loss: 0.2621 - val_loss: 0.2690
Epoch 12/1000
 - 4s - loss: 0.2541 - val_loss: 0.2291
Epoch 13/1000
 - 5s - loss: 0.2481 - val_loss: 0.2424
Epoch 14/1000
 - 5s - loss: 0.2404 - val_loss: 0.2273
Epoch 15/1000
 - 5s - loss: 0.2355 - val_loss: 0.2536
Epoch 16/1000
 - 5s - loss: 0.2322 - val_loss: 0.2249
Epoch 17/1000
 - 4s - loss: 0.2289 - val_loss: 0.2240
Epoch 18/1000
 - 5s - loss: 0.2262 - val

In [24]:
model.load_weights('R3_model.hdf5') # load weights from best model

In [25]:
from sklearn import metrics
import tensorflow as tf

pred = model.predict(x_test)
print(pred[0:5]) # print first five predictions

[[0.39993858]
 [0.04938626]
 [1.0200776 ]
 [1.8141118 ]
 [0.20206189]]


In [26]:
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print(f"Before save score (RMSE): {score}")

Before save score (RMSE): 0.4584360718727112


In [27]:
# save entire network to HDF5 (save everything, suggested)
model.save(os.path.join(path,"R3_model.h5"))

In [28]:
# save neural network structure to JSON (no weights)
model_json = model.to_json()
with open(os.path.join(path,"R3_model.json"), "w") as json_file:
    json_file.write(model_json)

In [29]:
# save neural network structure to YAML (no weights)
model_yaml = model.to_yaml()
with open(os.path.join(path,"R3_model.yaml"), "w") as yaml_file:
    yaml_file.write(model_yaml)